# Cleaning the Test Data

- I'm just going to quick clean up the test data that same way I did for the training data to be used later to make my predictions and submissiions. 

In [26]:
# The code was removed by Watson Studio for sharing.

In [27]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
import types
from ibm_botocore.client import Config
import ibm_boto3

warnings.filterwarnings('ignore')

%matplotlib inline

In [28]:
# The code was removed by Watson Studio for sharing.

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [29]:
df.shape

(1459, 80)

In [30]:
# Change YearBuilt into decades in order to plot it out easier

def construction_decade(row):
    if row['YearBuilt'] >= 1870 and row['YearBuilt'] < 1880:
        return '1870'
    elif row['YearBuilt'] >= 1880 and row['YearBuilt'] < 1890:
        return '1880'
    elif row['YearBuilt'] >= 1890 and row['YearBuilt'] < 1900:
        return '1890'
    elif row['YearBuilt'] >= 1900 and row['YearBuilt'] < 1910:
        return '1900'
    elif row['YearBuilt'] >= 1910 and row['YearBuilt'] < 1920:
        return '1910'
    elif row['YearBuilt'] >= 1920 and row['YearBuilt'] < 1930:
        return '1920'
    elif row['YearBuilt'] >= 1930 and row['YearBuilt'] < 1940:
        return '1930'
    elif row['YearBuilt'] >= 1940 and row['YearBuilt'] < 1950:
        return '1940'
    elif row['YearBuilt'] >= 1950 and row['YearBuilt'] < 1960:
        return '1950'
    elif row['YearBuilt'] >= 1960 and row['YearBuilt'] < 1970:
        return '1960'
    elif row['YearBuilt'] >= 1970 and row['YearBuilt'] < 1980:
        return '1970'
    elif row['YearBuilt'] >= 1980 and row['YearBuilt'] < 1990:
        return '1980'
    elif row['YearBuilt'] >= 1990 and row['YearBuilt'] < 2000:
        return '1990'
    elif row['YearBuilt'] >= 2000 and row['YearBuilt']:
        return '2000'
    else:
        return '0'
    
df['DecadeBuilt'] = df.apply(lambda row: construction_decade(row), axis=1)

In [31]:
df['DecadeBuilt'].head()

0    1960
1    1950
2    1990
3    1990
4    1990
Name: DecadeBuilt, dtype: object

## Fix Any Missing Data

In [32]:
# Find the percentage of missing values in each column
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageFinish,78,0.053461
GarageYrBlt,78,0.053461
GarageCond,78,0.053461
GarageQual,78,0.053461


In [33]:
# Fill in NaN values with "None" od "0"
df['PoolQC'].fillna('None', inplace=True)
df['MiscFeature'].fillna('None', inplace=True)
df['Alley'].fillna('None', inplace=True)
df['Fence'].fillna('None', inplace=True)
df['FireplaceQu'].fillna('None', inplace=True)
df['GarageYrBlt'].fillna('0', inplace=True)
df['BsmtExposure'].fillna('No', inplace=True)
df['MasVnrArea'].fillna('0', inplace=True)
df['MasVnrType'].fillna('None', inplace=True)
df['MasVnrArea'].astype('int', inplace=True).head()

0      0
1    108
2      0
3     20
4      0
Name: MasVnrArea, dtype: int64

In [34]:
# All NAN values in these 4 garage variables infers there is no garage. Change NAN to None. 
for col in ('GarageQual', 'GarageType', 'GarageCond', 'GarageFinish', 'GarageType'):
    df[col] = df[col].fillna('None')
    
# Basement variables with 'NAN' most likey don't have basements. Change NAN to None.
for col in ('BsmtFinType2','BsmtQual','BsmtCond','BsmtFinType1'):
    df[col] = df[col].fillna('None')

In [35]:
# NA values must be actual missing data. Group by neighborhood and impute the median LotFrontage.
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Fill the missing value with the mode.
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)

In [36]:
# Check to make sure there are no missing values left.
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
MSZoning,4,0.002742
Functional,2,0.001371
Utilities,2,0.001371
BsmtHalfBath,2,0.001371
BsmtFullBath,2,0.001371
KitchenQual,1,0.000685
BsmtFinSF1,1,0.000685
Exterior1st,1,0.000685
Exterior2nd,1,0.000685
GarageCars,1,0.000685


### There are 15 more columns with NaN values in the Test data then there was in the Training data. I'll have to go through all of these columns now.

#### MSZoning

In [37]:
# Find the unique values
df['MSZoning'].unique()

array(['RH', 'RL', 'RM', 'FV', 'C (all)', nan], dtype=object)

In [38]:
# Fill in NaN values with "None"
df['MSZoning'].fillna('None', inplace=True)

In [39]:
# Check
df['MSZoning'].unique()

array(['RH', 'RL', 'RM', 'FV', 'C (all)', 'None'], dtype=object)

#### Functional

In [40]:
df['Functional'].unique()

array(['Typ', 'Min2', 'Min1', 'Mod', 'Maj1', 'Sev', 'Maj2', nan], dtype=object)

In [41]:
# Fill NaN with "None"
df['Functional'].fillna('None', inplace=True)

In [42]:
#Check
df['Functional'].unique()

array(['Typ', 'Min2', 'Min1', 'Mod', 'Maj1', 'Sev', 'Maj2', 'None'], dtype=object)

#### Utilities

In [43]:
df['Utilities'].unique()

array(['AllPub', nan], dtype=object)

In [44]:
# Fill NaN with 'NonPub'
df['Utilities'].fillna('NonPub', inplace=True)

In [45]:
# Check.
df['Utilities'].unique()

array(['AllPub', 'NonPub'], dtype=object)

#### BsmtHalfBath

In [46]:
df['BsmtHalfBath'].unique()

array([  0.,   1.,  nan,   2.])

In [47]:
# Fill NaN values with '0'
df['BsmtHalfBath'].fillna('0', inplace=True)

In [53]:
# Change dtype from a float to an integer.
df['BsmtHalfBath'] = df['BsmtHalfBath'].astype('int', inplace=True)

In [54]:
df['BsmtHalfBath'].unique()

array([0, 1, 2])

#### BsmtFullBath

In [55]:
df['BsmtFullBath'].unique()

array([  0.,   1.,   2.,   3.,  nan])

In [56]:
# FIll NaN with '0'
df['BsmtFullBath'].fillna('0', inplace=True)

In [57]:
# Change dtype from a float to an integer
df['BsmtFullBath'] = df['BsmtFullBath'].astype('int', inplace=True)

In [58]:
df['BsmtFullBath'].unique()

array([0, 1, 2, 3])

#### KitchenQual

In [59]:
df['KitchenQual'].unique()

array(['TA', 'Gd', 'Ex', 'Fa', nan], dtype=object)

In [65]:
# Fill NaN values with 'None'
df['KitchenQual'].fillna('None', inplace=True)

In [66]:
# Check
df['KitchenQual'].unique()

array(['TA', 'Gd', 'Ex', 'Fa', 'None'], dtype=object)

#### BsmtFinSF1

In [77]:
df['BsmtFinSF1'].unique()

array([  468.,   923.,   791.,   602.,   263.,     0.,   935.,   637.,
         804.,  1051.,   156.,   300.,   514.,   110.,    28.,  1373.,
         578.,    24.,    16.,   326.,  1414.,   126.,   250.,  1129.,
        1298.,   280.,   368.,   376.,   466.,   244.,  1032.,   484.,
         833.,   506.,  1137.,   687.,   329.,   698.,  1059.,  1010.,
        1500.,   670.,   944.,  1188.,   856.,   936.,   734.,   339.,
         648.,   532.,   481.,   588.,   717.,    48.,   579.,   274.,
         780.,   176.,   283.,   788.,   474.,   188.,   452.,   264.,
         360.,   276.,   448.,   960.,   766.,  1026.,    73.,   736.,
         704.,   240.,   775.,  1319.,   267.,  1092.,   964.,   288.,
         104.,   192.,   954.,  1346.,  1433.,   860.,   870.,   353.,
         198.,   480.,  1682.,   672.,  1070.,   528.,   133.,   238.,
         426.,   375.,   343.,   747.,    76.,   308.,   615.,   679.,
          78.,    42.,   915.,   294.,   469.,   207.,   458.,   476.,
      

In [79]:
# Fill NaN value with '0'
df['BsmtFinSF1'].fillna('0', inplace=True)

In [80]:
# CHange from a float to an integer
df['BsmtFinSF1'] = df['BsmtFinSF1'].astype('int', inplace=True)

In [81]:
# Check
df['BsmtFinSF1'].unique()

array([ 468,  923,  791,  602,  263,    0,  935,  637,  804, 1051,  156,
        300,  514,  110,   28, 1373,  578,   24,   16,  326, 1414,  126,
        250, 1129, 1298,  280,  368,  376,  466,  244, 1032,  484,  833,
        506, 1137,  687,  329,  698, 1059, 1010, 1500,  670,  944, 1188,
        856,  936,  734,  339,  648,  532,  481,  588,  717,   48,  579,
        274,  780,  176,  283,  788,  474,  188,  452,  264,  360,  276,
        448,  960,  766, 1026,   73,  736,  704,  240,  775, 1319,  267,
       1092,  964,  288,  104,  192,  954, 1346, 1433,  860,  870,  353,
        198,  480, 1682,  672, 1070,  528,  133,  238,  426,  375,  343,
        747,   76,  308,  615,  679,   78,   42,  915,  294,  469,  207,
        458,  476, 1341,  564,  844,  847,  850,  284, 1965,  341,  741,
        189,  600,  400,  363,  832,  622,  225, 1333,  888,  636,  500,
        726,  254,  306,  435,  389,  320,  279,  536,  644, 1360,  986,
       1232, 2288, 1531, 1230, 1015, 1037, 1142, 12

#### TotalBsmtSF	

In [83]:
df['TotalBsmtSF'].unique()

array([  882.,  1329.,   928.,   926.,  1280.,   763.,  1168.,   789.,
        1300.,  1405.,   483.,   525.,   855.,   836.,  1590.,  1544.,
        1698.,  1822.,  2846.,  1671.,  1370.,  1324.,  1145.,   384.,
         847.,  1629.,  1595.,  1218.,  1468.,   831.,   816.,  1208.,
        1231.,  1390.,  1488.,   840.,   600.,   756.,   530.,  1642.,
         975.,  1492.,  1829.,  1224.,   715.,  1232.,  1209.,  1510.,
         533.,  1728.,  1140.,   782.,  1080.,  1604.,  1480.,  1143.,
        1206.,  1244.,   832.,   864.,   972.,   988.,   576.,  1175.,
        1395.,   709.,   936.,  1347.,   827.,  1027.,  1008.,   678.,
         930.,   346.,   738.,  1108.,   765.,   608.,   572.,   835.,
         780.,   528.,  1124.,   888.,   662.,  1422.,   689.,  1212.,
         707.,   856.,  1832.,  1319.,   720.,  1420.,   660.,  1117.,
         663.,  1172.,  1508.,  1250.,  1433.,   946.,  1222.,   676.,
         870.,   878.,  1026.,   876.,  1978.,  1040.,   848.,     0.,
      

In [84]:
df['TotalBsmtSF'].fillna('0', inplace=True)

In [85]:
df['TotalBsmtSF'] = df['TotalBsmtSF'].astype('int', inplace=True)

In [86]:
# Check
df['TotalBsmtSF'].unique()

array([ 882, 1329,  928,  926, 1280,  763, 1168,  789, 1300, 1405,  483,
        525,  855,  836, 1590, 1544, 1698, 1822, 2846, 1671, 1370, 1324,
       1145,  384,  847, 1629, 1595, 1218, 1468,  831,  816, 1208, 1231,
       1390, 1488,  840,  600,  756,  530, 1642,  975, 1492, 1829, 1224,
        715, 1232, 1209, 1510,  533, 1728, 1140,  782, 1080, 1604, 1480,
       1143, 1206, 1244,  832,  864,  972,  988,  576, 1175, 1395,  709,
        936, 1347,  827, 1027, 1008,  678,  930,  346,  738, 1108,  765,
        608,  572,  835,  780,  528, 1124,  888,  662, 1422,  689, 1212,
        707,  856, 1832, 1319,  720, 1420,  660, 1117,  663, 1172, 1508,
       1250, 1433,  946, 1222,  676,  870,  878, 1026,  876, 1978, 1040,
        848,    0,  381, 1196,  744,  960, 1107,  894, 1528, 1032,  423,
        982, 1191,  480,  629, 1594, 1049, 1243, 1958, 1336, 1214, 1068,
       1430, 1344,  945,  546,  833, 1216,  624, 1679, 1152, 1434, 2208,
       1418, 1587,  952, 1181, 1122, 1427, 1043, 10

#### BsmtUnfSF

In [87]:
df['BsmtUnfSF'].unique()

array([  270.,   406.,   137.,   324.,  1017.,   763.,   233.,   789.,
         663.,     0.,   354.,   327.,   225.,   341.,   836.,  1590.,
        1544.,  1588.,  1794.,  1473.,  1093.,  1346.,  1324.,  1129.,
          58.,   847.,  1629.,  1595.,  1218.,    54.,   831.,   816.,
         223.,   261.,   190.,   840.,   320.,   224.,   290.,   286.,
         610.,   491.,   659.,  1323.,   143.,   386.,   534.,   150.,
         500.,   533.,   228.,   470.,   482.,   136.,  1604.,   292.,
         125.,   510.,   832.,   525.,   456.,   576.,   161.,   237.,
         415.,   661.,   357.,   827.,   247.,   678.,   930.,   346.,
         455.,   200.,   634.,   577.,   144.,   308.,   316.,   480.,
         252.,   164.,   888.,   780.,   662.,   656.,   689.,   186.,
         232.,   120.,  1128.,    89.,   328.,   660.,   153.,   172.,
         564.,   218.,   162.,  1250.,    86.,  1198.,   676.,   140.,
         396.,   296.,  1040.,   176.,   381.,   126.,   744.,   960.,
      

In [88]:
df['BsmtUnfSF'].fillna('0', inplace=True)

In [89]:
df['BsmtUnfSF'] = df['BsmtUnfSF'].astype('int', inplace=True)

In [90]:
df['BsmtUnfSF'].unique()

array([ 270,  406,  137,  324, 1017,  763,  233,  789,  663,    0,  354,
        327,  225,  341,  836, 1590, 1544, 1588, 1794, 1473, 1093, 1346,
       1324, 1129,   58,  847, 1629, 1595, 1218,   54,  831,  816,  223,
        261,  190,  840,  320,  224,  290,  286,  610,  491,  659, 1323,
        143,  386,  534,  150,  500,  533,  228,  470,  482,  136, 1604,
        292,  125,  510,  832,  525,  456,  576,  161,  237,  415,  661,
        357,  827,  247,  678,  930,  346,  455,  200,  634,  577,  144,
        308,  316,  480,  252,  164,  888,  780,  662,  656,  689,  186,
        232,  120, 1128,   89,  328,  660,  153,  172,  564,  218,  162,
       1250,   86, 1198,  676,  140,  396,  296, 1040,  176,  381,  126,
        744,  960,  974,  618,  894,  850,  657,   80,  235, 1115,  321,
        756,  221,  434, 1958, 1258, 1214,  384, 1026, 1430, 1344,   30,
        370,  546,  395,   36, 1216,  166,  388,  208,  318,  590,   98,
        158,  100,  243, 1077,  846,  195,  476,  1

#### SaleType

In [91]:
df['SaleType'].unique()

array(['WD', 'COD', 'New', 'ConLD', 'Oth', 'Con', 'ConLw', 'ConLI', 'CWD',
       nan], dtype=object)

In [92]:
df['SaleType'].fillna('None', inplace=True)

In [93]:
df['SaleType'].unique()

array(['WD', 'COD', 'New', 'ConLD', 'Oth', 'Con', 'ConLw', 'ConLI', 'CWD',
       'None'], dtype=object)

#### BsmtFinSF2

In [95]:
df['BsmtFinSF2'].unique()

array([  144.,     0.,    78.,   859.,   981.,    42.,    46.,   162.,
         174.,   350.,   263.,  1073.,    12.,   159.,   474.,   453.,
         684.,   387.,   688.,   252.,   590.,   284.,   276.,   168.,
         294.,   622.,   113.,  1526.,   360.,   774.,   364.,   596.,
         884.,    92.,   216.,   136.,   147.,   201.,   512.,   247.,
         202.,   483.,   750.,    60.,   102.,    95.,    63.,   262.,
         393.,   286.,   450.,    72.,   243.,   694.,   875.,   507.,
         435.,   419.,   250.,   116.,   624.,    76.,   270.,   110.,
         288.,   186.,   449.,    48.,   613.,   495.,   852.,   555.,
         799.,   811.,   842.,   670.,   382.,   456.,   308.,    52.,
         196.,   128.,   488.,   319.,   273.,    nan,   240.,   956.,
         723.,   120.,   679.,    40.,   121.,   604.,   153.,   619.,
           6.,   351.,  1037.,   829.,   264.,    38.,   206.,   167.,
         543.,   259.,   404.,   138.,   955.,   691.,   354.,    66.,
      

In [96]:
df['BsmtFinSF2'].fillna('0', inplace=True)

In [98]:
df['BsmtFinSF2'] = df['BsmtFinSF2'].astype('int', inplace=True)

#### GarageCars

In [99]:
df['GarageCars'].unique()

array([  1.,   2.,   3.,   0.,   4.,   5.,  nan])

In [100]:
df['GarageCars'].fillna('0', inplace=True)

In [101]:
df['GarageCars'] = df['GarageCars'].astype('int', inplace=True)

#### GarageArea

In [102]:
df['GarageArea'].unique()

array([  730.,   312.,   482.,   470.,   506.,   440.,   420.,   393.,
         525.,   511.,   264.,   320.,   308.,   751.,   868.,   678.,
         958.,   756.,   484.,   430.,   437.,   400.,   433.,   962.,
         880.,   676.,   528.,   429.,   461.,   762.,   713.,   588.,
         480.,   496.,   852.,   592.,   596.,   535.,   660.,   441.,
         490.,   504.,   517.,     0.,   315.,   576.,   620.,   263.,
         318.,   305.,   280.,   240.,   551.,   780.,   288.,   624.,
         363.,   200.,   572.,   180.,   365.,   231.,   450.,   416.,
         591.,   690.,   522.,   467.,   209.,   366.,  1017.,   615.,
         580.,   513.,   850.,   621.,   319.,   336.,   352.,   260.,
         399.,   473.,   598.,   531.,   625.,   330.,   741.,   776.,
         520.,   686.,   253.,   286.,   495.,   616.,   275.,   502.,
         632.,   390.,   564.,   396.,   409.,   567.,   529.,   479.,
         581.,   565.,   478.,   477.,   345.,   656.,   754.,   864.,
      

In [103]:
df['GarageArea'].fillna('0', inplace=True)

In [105]:
df['GarageArea'] = df['GarageArea'].astype('int', inplace=True)

#### Exterior1st

In [106]:
df['Exterior1st'].unique()

array(['VinylSd', 'Wd Sdng', 'HdBoard', 'Plywood', 'MetalSd', 'CemntBd',
       'WdShing', 'BrkFace', 'AsbShng', 'BrkComm', 'Stucco', 'AsphShn',
       nan, 'CBlock'], dtype=object)

In [107]:
df['Exterior1st'].fillna('Unknown', inplace=True)

#### Exterior2nd

In [108]:
df['Exterior2nd'].unique()

array(['VinylSd', 'Wd Sdng', 'HdBoard', 'Plywood', 'MetalSd', 'Brk Cmn',
       'CmentBd', 'ImStucc', 'Wd Shng', 'AsbShng', 'Stucco', 'CBlock',
       'BrkFace', 'AsphShn', nan, 'Stone'], dtype=object)

In [109]:
df['Exterior2nd'].fillna('Exterior2nd', inplace=True)

In [119]:
# Add a new column for total square feet.
df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']

In [120]:
df['TotalSF'].head()

0    1778
1    2658
2    2557
3    2530
4    2560
Name: TotalSF, dtype: int64

In [110]:
# Check to make sure there are no missing values left.
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
DecadeBuilt,0,0.0
Heating,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
MasVnrArea,0,0.0
ExterQual,0,0.0
ExterCond,0,0.0


## Data is all cleaned up and there are no more missing values. Time to Encode and get dummy variables. 

## Encoding
- Changing categorical features into numeric features to be used in models.
- If I felt like the columns features could be ranked from worst to beat I will use a label encoder.
- If I felt like the columns features could not be ranked I will use getdummies. 

In [111]:
# Change any columns of strings in a panda's dataframe to a column of
# categorical values. This applies the changes inplace
def train_cats(df):
     for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

In [112]:
train_cats(df)

### Ordering all the same columns I did for the training data in the EDA notebook

In [113]:
df.Street.cat.set_categories((['Grvl', 'Pave']), ordered=True, inplace=True)
df.LandSlope.cat.set_categories((['Sev', 'Mod', 'Gtl']), ordered=True, inplace=True)
df.ExterQual.cat.set_categories((['Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.ExterCond.cat.set_categories((['Po', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.HeatingQC.cat.set_categories((['Po', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.CentralAir.cat.set_categories((['N', 'Y']), ordered=True, inplace=True)
df.KitchenQual.cat.set_categories((['Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.FireplaceQu.cat.set_categories((['None', 'Po', 'TA', 'Fa', 'Gd', 'Ex']), ordered=True, inplace=True)
df.GarageFinish.cat.set_categories((['None', 'Unf', 'RFn', 'Fin']), ordered=True, inplace=True)
df.GarageQual.cat.set_categories((['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.GarageCond.cat.set_categories((['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.PavedDrive.cat.set_categories((['N', 'P', 'Y']), ordered=True, inplace=True)
df.BsmtQual.cat.set_categories((['None', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.BsmtCond.cat.set_categories((['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']), ordered=True, inplace=True)
df.BsmtExposure.cat.set_categories((['No', 'Av', 'Gd', 'Mn']), ordered=True, inplace=True)
df.PoolQC.cat.set_categories((['None', 'Fa', 'Gd', 'Ex']), ordered=True, inplace=True)

## Label Encoding

In [114]:
label_cols = ('Street', 'Alley', 'LandSlope', 'ExterQual', 'ExterCond', 'HeatingQC', 
              'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageQual', 
              'GarageCond', 'PavedDrive', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
              'PoolQC')

for c in label_cols:
    le = LabelEncoder()
    le.fit(list(df[c].values))
    df[c] = le.transform(list(df[c].values))

## Get Dummy Variables
- Get dummy variables for the rest of the columns.

In [115]:
dummy_df = pd.get_dummies(df, columns=['MSZoning', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood',
            'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
            'BsmtExposure', 'BsmtFinType1', 'Heating','Electrical', 'GarageType', 'SaleType', 
            'Condition1', 'Condition2', 'HouseStyle', 'BsmtFinType2', 'Functional', 'SaleCondition',
            'Foundation', 'Heating', 'Fence', 'MiscFeature'], drop_first=True)

In [117]:
# Function that displays all columns and rows when you use commands like df.head() and df_tail().

def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [121]:
display_all(dummy_df.head().transpose())

,0,1,2,3,4
Id,1461,1462,1463,1464,1465
MSSubClass,20,20,60,60,120
LotFrontage,80,81,74,78,43
LotArea,11622,14267,13830,9978,5005
Street,1,1,1,1,1
Alley,1,1,1,1,1
LandSlope,0,0,0,0,0
OverallQual,5,6,5,6,8
OverallCond,6,6,5,6,5
YearBuilt,1961,1958,1997,1998,1992


In [122]:
# Save dataframe as csv file to storage
project.save_data(data=dummy_df.to_csv(index=False),file_name='clean_test.csv',overwrite=True)

{'asset_id': 'a2460d11-2f84-42f8-9790-24da3ec3cd3d',
 'bucket_name': 'housepricesadvancedregressiontech-donotdelete-pr-xspbt29nepkblo',
 'file_name': 'clean_test.csv',
 'message': 'File saved to project storage.'}